In [1]:
import os
import sys
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

Using TensorFlow backend.


In [2]:
BASE_DIR = r'D:\Work\DS_and_AI\Data_Sets'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
print(GLOVE_DIR)
#MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


D:\Work\DS_and_AI\Data_Sets\glove.6B


In [3]:
data = pd.read_csv('train.csv',)

In [4]:
data

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [5]:
data.drop(['qid'],axis=1,inplace=True)

In [6]:
#data

In [7]:
#format our text samples and labels into tensors that can be fed into a neural network
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['question_text'])
sequences = tokenizer.texts_to_sequences(data['question_text'])

In [8]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 222161 unique tokens.


In [9]:
#finding the max_elnth of the sequence
max_len = 0
i= 0 
for i in range(len(sequences)):
    if len(sequences[i]) > max_len:
        max_len = len(sequences[i])
        #print(max_len)
        #print(sequences[i])
print(max_len)
        
    

190


In [10]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_data = pad_sequences(sequences, maxlen=max_len)


Found 222161 unique tokens.


In [11]:

labels = to_categorical(np.asarray(data['target']))
#labels = pd.DataFrame(labels)
#labels[0:24]

print(labels.shape)
print(X_data.shape)

(1306122, 2)
(1306122, 190)


In [12]:
# split the data into a training set and a validation set
indices = np.arange(X_data.shape[0])
np.random.shuffle(indices)
X_data = X_data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_data.shape[0])
print(nb_validation_samples)

x_train = X_data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

261224


In [13]:
x_train

array([[    0,     0,     0, ...,    46, 17691,  5801],
       [    0,     0,     0, ...,   338,     3,  5006],
       [    0,     0,     0, ...,     4,   366,   214],
       ...,
       [    0,     0,     0, ...,  5999,    10,  6364],
       [    0,     0,     0, ...,   168,     4,   241],
       [    0,     0,     0, ...,  5502,  2894,   121]])

In [14]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [15]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size = 0.2, random_state = 0)

In [16]:
#preparing the embedding layer
embeddings_index = {}
f = os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')
f = open(f,encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [17]:
#At this point we can leverage our embedding_index dictionary and our word_index to compute our embedding matrix:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [18]:
embedding_matrix.shape

(222162, 100)

In [19]:
#We load this embedding matrix into an Embedding layer. Note that we set trainable=False to prevent the weights from being updated during training.
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [20]:
#Finally we can then build a small 1D convnet to solve our classification problem:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
#x = Conv1D(128, 5, activation='relu')(x)
#x = MaxPooling1D(5)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(2, activation='sigmoid')(x)

model = Model(sequence_input, outputs)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
#fitting
model.fit(x_train, y_train, validation_data=(x_val,y_val),
          epochs=2, batch_size=128)

Train on 1044898 samples, validate on 261224 samples
Epoch 1/2
1044898/1044898 [==============================] - 2648s 3ms/step - loss: 0.2018 - acc: 0.9427 - val_loss: 0.1992 - val_acc: 0.9438
Epoch 2/2
1044898/1044898 [==============================] - 2638s 3ms/step - loss: 0.1976 - acc: 0.9438 - val_loss: 0.1970 - val_acc: 0.9439


In [21]:
y_pred = model.predict(x_val)

In [22]:
from  sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_val,y_pred)

In [23]:
auc

0.6867251656320031

In [24]:
pred = pd.DataFrame(y_pred)


In [25]:
pred.to_csv('predctions.csv',index=False)

In [26]:
model.save_weights('prj_3_1D_CNN.h5')